In [1]:
import os, re, typing, jieba, pypinyin

In [3]:
filename = "/home/cxn/BZNSYP/000001-010000.txt"
with open(filename, 'r', encoding='utf-8') as f:
	lines = f.readlines()

In [10]:
regex = re.compile('#\d+')

In [18]:
dataset: typing.List[typing.Tuple[str, str]]  = []
for i in range(0, len(lines), 2):
	splitpos = lines[i].index('\t')
	text = lines[i][splitpos+1:].strip('\n')
	text = regex.sub("", text)

	phome = lines[i + 1].strip('\n').strip()
	dataset.append((text, phome))
print(f"Got dataset {len(dataset)}")


Got dataset 10000


In [19]:
dataset[:5]

[('卡尔普陪外孙玩滑梯。', 'ka2 er2 pu3 pei2 wai4 sun1 wan2 hua2 ti1'),
 ('假语村言别再拥抱我。', 'jia2 yu3 cun1 yan2 bie2 zai4 yong1 bao4 wo3'),
 ('宝马配挂跛骡鞍，貂蝉怨枕董翁榻。',
  'bao2 ma3 pei4 gua4 bo3 luo2 an1 diao1 chan2 yuan4 zhen3 dong3 weng1 ta4'),
 ('邓小平与撒切尔会晤。', 'deng4 xiao3 ping2 yu3 sa4 qie4 er3 hui4 wu4'),
 ('老虎幼崽与宠物犬玩耍。', 'lao2 hu3 you4 zai3 yu2 chong3 wu4 quan3 wan2 shua3')]

In [4]:
token_filename = "assets/dataset/tokens.txt"
token_set: typing.Set[str] = set()
with open(token_filename, 'r', encoding='utf-8') as f:
	for line in f.readlines():
		token = line.strip('\n').strip().split(' ')[0]
		token_set.add(token)
print(f"Got token set {len(token_set)}")

Got token set 2069


In [24]:
baker_token_set = set()
for (text, phome) in dataset:
	for token in phome.split(' '):
		baker_token_set.add(token)
print(f"Got baker token set {len(baker_token_set)}")

Got baker token set 1607


In [64]:
diff_set = token_set - baker_token_set
print(f"Got diff set {len(diff_set)}")

Got diff set 768


In [65]:
maps: typing.Dict[str, typing.Set[int]] = { x: set() for x in token_set } # { token: [line number] }
abandon_token: typing.Set[str] = set()
for i, (text, phome) in enumerate(dataset):
	for token in phome.split(' '):
		if token not in token_set:
			abandon_token.add(token)
		else:
			maps[token].add(i)
print(f"Abandon tokens:", abandon_token)

Abandon tokens: {'chi5', 'jun5', 'hao5', 'wang5', 'pa5', 'cong5', 'nao5', 'luo5', 'xie5', 'guo5', 'mao5', 'qiu5', 'guang5', 'jingr3', 'tun5', 'sai5', 'renr4', 'yanr3', 'lo5', 'wu5', 'zhuan5', 'lei5', 'bang5', 'tianr1', 'qiang5', 'nar4', 'lian5', 'shang5', 'fen5', 'yangr4', 'nang5', 'gong5', 'lang5', 'jian5', 'mai5', 'tui5', 'nv5', 'yao5', 'hu5', 'dan5', 'sheng5', 'dianr2', 'fan5', 'lai5', 'chuan5', 'pai5', 'yanr2', 'lv5', 'tuo5', 'mei5', 'wen5', 'shu5', 'liar3', 'chao5', 'di5', 'du5', 'wei5', 'dianr3', 'han5', 'duor3', 'ter4', 'cha5', 'huan5', 'hou5', 'xiao5', 'mo5', 'lu5', 'peng5', 'dunr3', 'mou5', 'e5', 'fu5', 'tao5', 'shei2', 'nar3', 'ma5', 'nai5', 'duir1', 'qin5', 'zai5', 'bu5', 'lanr4', 'he5', 'rang5', 'zao5', 'huir3', 'chen5', 'lie5', 'ying5', 'pian5', 'huir4', 'zhi5', 'huang5', 'xin5', 'yi5', 'quan5', 'menr5', 'yuan5', 'tan5', 'chu5', 'weir4', 'wanr1', 'sao5', 'run5', 'tei1', 'dao5', 'duir4', 'zong5', 'la5', 'tong5', 'me5', 'li5', 'shi5', 'sou5', 'shao5', 'gou5', 'chui5', 'su5',

In [66]:
freq = [ (key, len(value)) for key, value in maps.items() ]
freq.sort(key=lambda x: x[1])
print(freq[:5])
print(freq[-5:])

[('wo', 0), ('na', 0), ('tun4', 0), ('ce', 0), ('nun2', 0)]
[('bu4', 1165), ('you3', 1180), ('zai4', 1428), ('yi4', 1653), ('shi4', 2512)]


In [67]:
agresive_maps: typing.Dict[str, typing.List[int]] = {} # token: [line number]
for (key, value) in maps.items():
	line_length: typing.List[typing.Tuple[int, int]] = [] # (line number, length)
	for line_num in value:
		line_length.append((line_num, len(dataset[line_num][1].split(' '))))
	line_length.sort(key=lambda x: x[1], reverse=True)
	agresive_maps[key] = [ x[0] for x in line_length ]

In [68]:
n_select = 3
select_set: typing.Set[int] = set()
for (token, count) in freq:
	for i in agresive_maps[token][:n_select]:
		select_set.add(i)
outputs = [ dataset[i] for i in select_set]
print(f"Got outputs: {len(outputs)}")

Got outputs: 1403


In [69]:
outputs[:5]

[('躺在急救担架上的男子双目紧闭，头发散发出一股烧焦的味道。',
  'tang3 zai4 ji2 jiu4 dan1 jia4 shang4 de5 nan2 zi3 shuang1 mu4 jin3 bi4 tou2 fa4 san4 fa1 chu1 yi4 gu3 shao1 jiao1 de5 wei4 dao4'),
 ('工业园区是承接产业转移、加速产业集聚、培育产业集群的主要载体。',
  'gong1 ye4 yuan2 qu1 shi4 cheng2 jie1 chan3 ye4 zhuan3 yi2 jia1 su4 chan3 ye4 ji2 ju4 pei2 yu4 chan3 ye4 ji2 qun2 de5 zhu3 yao4 zai4 ti3'),
 ('那一刻，我才真正的懂你，就像懂我现在的自己。',
  'na4 yi2 ke4 wo3 cai2 zhen1 zheng4 de5 dong2 ni3 jiu4 xiang4 dong2 wo3 xian4 zai4 de5 zi4 ji3'),
 ('由于列车长时间停靠，车厢内的空气越来越“闷”。',
  'you2 yu2 lie4 che1 zhang3 shi2 jian1 ting2 kao4 che1 xiang1 nei4 de5 kong1 qi4 yue4 lai2 yue4 men1'),
 ('但如果按车队规模，一嗨数千辆车的量级绝对算不上最大。',
  'dan4 ru2 guo3 an4 che1 dui4 gui1 mo2 yi4 hai1 shu4 qian1 liang4 che1 de5 liang4 ji2 jue2 dui4 suan4 bu2 shang4 zui4 da4')]

In [71]:
mandarin_file = "assets/text/mandarin.txt"
mandarin_dataset: typing.List[typing.Tuple[str, str]] = []
with open(mandarin_file, 'r', encoding='utf-8') as f:
	for i, line in enumerate(f.readlines()):
		text = line.strip('\n').strip()
		normalized = text.replace('，',',').replace('。', '.').replace('？','?').replace('！','!').replace('、', ',').replace('”','"').replace('“','"')
		cuts = list(jieba.cut(normalized))
		phome = []
		for cut in cuts:
			if cut == ',' or cut == '.' or cut == '?' or cut == '!' or cut == '"':
				continue
			pinyins = pypinyin.lazy_pinyin(cut, style=pypinyin.Style.TONE3, tone_sandhi=True)
			for pinyin in pinyins:
				phome.append(pinyin)
		phome = ' '.join(phome)
		mandarin_dataset.append((text, phome))
print(f"Got mandarin: {len(mandarin_dataset)}")

Got mandarin: 729


In [72]:
mandarin_dataset[:5]

[('他在家里吃苹果，喝茶，打电话，唱歌。',
  'ta1 zai4 jia1 li3 chi1 ping2 guo3 he1 cha2 da3 dian4 hua4 chang4 ge1'),
 ('我去商店买水果和书，看到小猫在玩。',
  'wo3 qu4 shang1 dian4 mai3 shui3 guo3 he2 shu1 kan4 dao4 xiao3 mao1 zai4 wan2'),
 ('小明去北京吃炸酱面，他在公园里看见了许多种类的鸟，甚至还遇到了一只大猩猩。',
  'xiao3 ming2 qu4 bei3 jing1 chi1 zha2 jiang4 mian4 ta1 zai4 gong1 yuan2 li3 kan4 jian4 le xu3 duo1 zhong3 lei4 de niao3 shen4 zhi4 hai2 yu4 dao4 le yi4 zhi1 da4 xing1 xing1'),
 ('小猫学会了跳舞，爬上了高高的树。',
  'xiao3 mao1 xue2 hui4 le tiao4 wu3 pa2 shang4 le gao1 gao1 de shu4'),
 ('风筝在天空中飞得又高又快，真是太漂亮了。',
  'feng1 zheng1 zai4 tian1 kong1 zhong1 fei1 de2 you4 gao1 you4 kuai4 zhen1 shi4 tai4 piao4 liang4 le')]

In [73]:
mandarin_maps: typing.Dict[str, typing.Set[int]] = { x: set() for x in token_set }
mandarin_abandon_token: typing.Set[str] = set()
for i, (text, phome) in enumerate(mandarin_dataset):
	for token in phome.split(' '):
		if token not in token_set:
			mandarin_abandon_token.add(token)
		else:
			mandarin_maps[token].add(i)
print(f"Abandon tokens:", mandarin_abandon_token)

Abandon tokens: {'5G', '—', 'AI', 'GDPR'}


In [75]:
dataset_has_phome: typing.Set[str] = set()
for token, line_nums in agresive_maps.items():
	if len(line_nums) > 0:
		dataset_has_phome.add(token)
print(f"dataset got {len(dataset_has_phome)} tokens.")
mandarin_dataset_has_phome: typing.Set[str] = set()
for token, line_nums in mandarin_maps.items():
	if len(line_nums) > 0:
		mandarin_dataset_has_phome.add(token)
print(f"mandarin got {len(mandarin_dataset_has_phome)} tokens.")

dataset got 1301 tokens.
mandarin got 780 tokens.


In [76]:
man_has_but_baker_not_has = mandarin_dataset_has_phome - dataset_has_phome
print(f"man_has_but_baker_not_has: {len(man_has_but_baker_not_has)}")

man_has_but_baker_not_has: 11


In [77]:
diff_list = list(man_has_but_baker_not_has)
diff_list[:5]

['huo', 'ma', 'er', 'zi', 'men']

In [79]:
mandarin_dataset[list(mandarin_maps["huo"])[0]]

('天气逐渐变冷，要注意穿暖和的衣服。',
 'tian1 qi4 zhu2 jian4 bian4 leng3 yao4 zhu4 yi4 chuan1 nuan3 huo de yi1 fu2')

In [80]:
final_output: typing.List[str] = []
for baker_text, baker_phomes in outputs:
	final_output.append(baker_text)
	final_output.append(baker_phomes)
for mandarin_token in diff_list:
	(text, phome) = mandarin_dataset[list(mandarin_maps[mandarin_token])[0]]
	final_output.append(text)
	final_output.append(phome)
print(f"lens: {len(final_output) / 2}")

lens: 1414.0


In [82]:
with open("assets/text/full_mandarin.txt", "w", encoding="utf-8") as f:
	f.write("\n".join(final_output))
print("done.")

done.


## Filter again

In [2]:
dataset: typing.List[typing.Tuple[str, str]] = []
with open("assets/text/full_mandarin.txt", "r", encoding="utf-8") as f:
	lines = f.readlines()
for i in range(0, len(lines), 2):
	text = lines[i].strip().strip('\n')
	phome = lines[i + 1].strip().strip('\n')
	dataset.append((text, phome))
print(f"len: {len(dataset)}")

len: 1414


In [3]:
dataset[:5]

[('躺在急救担架上的男子双目紧闭，头发散发出一股烧焦的味道。',
  'tang3 zai4 ji2 jiu4 dan1 jia4 shang4 de5 nan2 zi3 shuang1 mu4 jin3 bi4 tou2 fa4 san4 fa1 chu1 yi4 gu3 shao1 jiao1 de5 wei4 dao4'),
 ('工业园区是承接产业转移、加速产业集聚、培育产业集群的主要载体。',
  'gong1 ye4 yuan2 qu1 shi4 cheng2 jie1 chan3 ye4 zhuan3 yi2 jia1 su4 chan3 ye4 ji2 ju4 pei2 yu4 chan3 ye4 ji2 qun2 de5 zhu3 yao4 zai4 ti3'),
 ('那一刻，我才真正的懂你，就像懂我现在的自己。',
  'na4 yi2 ke4 wo3 cai2 zhen1 zheng4 de5 dong2 ni3 jiu4 xiang4 dong2 wo3 xian4 zai4 de5 zi4 ji3'),
 ('由于列车长时间停靠，车厢内的空气越来越“闷”。',
  'you2 yu2 lie4 che1 zhang3 shi2 jian1 ting2 kao4 che1 xiang1 nei4 de5 kong1 qi4 yue4 lai2 yue4 men1'),
 ('但如果按车队规模，一嗨数千辆车的量级绝对算不上最大。',
  'dan4 ru2 guo3 an4 che1 dui4 gui1 mo2 yi4 hai1 shu4 qian1 liang4 che1 de5 liang4 ji2 jue2 dui4 suan4 bu2 shang4 zui4 da4')]

In [4]:
token_filename = "assets/dataset/tokens.txt"
token_set: typing.Set[str] = set()
with open(token_filename, 'r', encoding='utf-8') as f:
	for line in f.readlines():
		token = line.strip('\n').strip().split(' ')[0]
		token_set.add(token)
print(f"Got token set {len(token_set)}")

Got token set 2069


In [5]:
maps: typing.Dict[str, typing.Set[int]] = { x: set() for x in token_set } # { token: [line number] }
abandon_token: typing.Set[str] = set()
for i, (text, phome) in enumerate(dataset):
	for token in phome.split(' '):
		if token not in token_set:
			abandon_token.add(token)
		else:
			maps[token].add(i)
print(f"Abandon tokens:", abandon_token)

Abandon tokens: {'kan5', 'jun5', 'jinr4', 'wu5', 'teng5', 'na5', 'suo5', 'niang5', 'lai5', 'ti5', 'la5', 'jia5', 'tao5', 'fen5', 'bo5', 'jing5', 'hair2', 'qi5', 'cha5', 'mei5', 'nai5', 'hua5', 'ju5', 'dianr3', 'die5', 'ba5', 'ji5', 'mi5', 'ger4', 'liang5', 'fur4', 'sha5', 'huir4', 'cao5', 'bang5', 'qu5', 'ma5', 'shao5', 'kuai5', 'shei2', 'gua5', 'ng1', 'bao5', 'ter4', 'xing5', 'bai5', 'shi5', 'fang5', 'cir2', 'zhe5', 'long5', 'zhi5', 'you5', 'pi5', 'heng5', 'dou5', 'tan5', 'ye5', 'zha5', 'hao5', 'menr5', 'qing5', 'da5', 'zu5', 'luo5', 'tu5', 'xi5', 'le5', 'du5', 'zhei4', 'tang5', 'zhou5', 'su5', 'shuo5', 'qin5', 'pan5', 'tuo5', 'yuan5', 'pai5', 'mao5', 'sao5', 'peng5', 'xie5', 'hu5', 'guo5', 'fu5', 'er5', 'lu5', 'kou5', 'niu5', 'gu5', 'jiu5', 'yang5', 'sou5', 'tou5', 'ya5', 'duo5', 'niur1', 'se5', 'qiu5', 'chan5', 'kuair4', 'sheng5', 'dao5', 'ha5', 'o5', 'yi5', 'rang5', 'tongr4', 'shuan5', 'a5', 'qie5', 'ne5', 'sa5', 'si5', 'li5', 'bu5', 'zi5', 'nan5', 'huan5', 'hou5', 'huo5', 'yir4', 

In [8]:
scores = [ 0 for _ in range(len(dataset))]
for i, (text, phome) in enumerate(dataset):
	for token in phome.split(' '):
		if token in maps:
			if len(maps[token]) == 1:
				scores[i] += 100
			elif len(maps[token]) == 2:
				scores[i] += 1
scores[-5:]

[2, 1, 100, 100, 100]

In [ ]:
agresive_maps: typing.Dict[str, typing.List[int]] = {} # token: [line number]
for (key, value) in maps.items():
	line_list = list(value)
	line_list.sort(key=lambda x: scores[x], reverse=True)
	agresive_maps[key] = line_list